In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'


import os, sys, random
from pathlib import Path
import glob

from tqdm import tqdm
from fastprogress import progress_bar

import warnings
warnings.filterwarnings(action='ignore')

import pickle

In [2]:
CUR_DIR = Path.cwd()
DATA_DIR = CUR_DIR / 'data/preprocessed'
DATA_DIR

WindowsPath('C:/Users/안현준/풍년보험/data/preprocessed')

In [3]:
crops = ['감자', '고구마', '고추(화건)', '깻잎', '녹두(국산)', '단감', '당근(무세척)', '대파', '들깨(국산)',
       '딸기', '무', '미나리', '방울토마토', '배(신고)', '배추', '백참깨(국산)', '복숭아(백도)',
       '사과(후지)', '생강(국산)', '수박', '시금치', '애호박', '양배추', '양파', '오이(가시)', '적상추',
       '참외', '콩(흰콩국산)', '토마토', '팥(국산)', '포도(캠벨)', '풋고추']
TARGET_YEAR = [2017, 2018, 2019, 2020]

In [4]:
df_total = pd.read_csv('./results/손해율검정.csv', encoding='utf-8')
df_total

,Unnamed: 0,검정연도,cluster_id,손해율(true),손해율(pred)
0,0,2017,0,0.473449,1.858337
1,1,2017,1,4.033635,0.107160
2,2,2017,2,1.540479,1.581532
3,3,2017,3,3.954650,4.781895
4,4,2017,4,3.738867,3.568686
5,5,2017,5,0.000000,0.438442
6,6,2017,6,4.276381,3.472149
7,0,2018,0,2.208625,1.265839
8,1,2018,1,1.218203,0.000000
9,2,2018,2,0.000000,1.367975


In [5]:
df_total = df_total[df_total.columns[1:]]
df_total

,검정연도,cluster_id,손해율(true),손해율(pred)
0,2017,0,0.473449,1.858337
1,2017,1,4.033635,0.107160
2,2017,2,1.540479,1.581532
3,2017,3,3.954650,4.781895
4,2017,4,3.738867,3.568686
5,2017,5,0.000000,0.438442
6,2017,6,4.276381,3.472149
7,2018,0,2.208625,1.265839
8,2018,1,1.218203,0.000000
9,2018,2,0.000000,1.367975


In [6]:
with open('./results/cluster_id.pkl', 'rb') as f:
    cluster_id = pickle.load(f)
cluster_id

defaultdict(list,
            {0: ['감자', '당근(무세척)', '무', '미나리', '배추', '양파'],
             4: ['고구마', '대파', '배(신고)', '백참깨(국산)', '사과(후지)', '생강(국산)', '팥(국산)'],
             3: ['고추(화건)', '녹두(국산)', '들깨(국산)', '콩(흰콩국산)'],
             2: ['깻잎', '딸기', '방울토마토', '수박', '애호박', '양배추', '오이(가시)', '토마토'],
             1: ['단감', '복숭아(백도)', '참외', '풋고추'],
             6: ['시금치', '적상추'],
             5: ['포도(캠벨)']})

In [7]:
cnt = 0

def pick_group(idx, items=[], n_pick=0):
    if n_pick == 3:
        global cnt
        cnt += 1
        print(items, cnt)
        return
    for i in range(idx + 1, 6 + 1):
        items.append(i)
        pick_group(i, items, n_pick + 1)
        items.pop()
        
pick_group(-1, items=[], n_pick=0)

[0, 1, 2] 1
[0, 1, 3] 2
[0, 1, 4] 3
[0, 1, 5] 4
[0, 1, 6] 5
[0, 2, 3] 6
[0, 2, 4] 7
[0, 2, 5] 8
[0, 2, 6] 9
[0, 3, 4] 10
[0, 3, 5] 11
[0, 3, 6] 12
[0, 4, 5] 13
[0, 4, 6] 14
[0, 5, 6] 15
[1, 2, 3] 16
[1, 2, 4] 17
[1, 2, 5] 18
[1, 2, 6] 19
[1, 3, 4] 20
[1, 3, 5] 21
[1, 3, 6] 22
[1, 4, 5] 23
[1, 4, 6] 24
[1, 5, 6] 25
[2, 3, 4] 26
[2, 3, 5] 27
[2, 3, 6] 28
[2, 4, 5] 29
[2, 4, 6] 30
[2, 5, 6] 31
[3, 4, 5] 32
[3, 4, 6] 33
[3, 5, 6] 34
[4, 5, 6] 35


In [8]:
df = df_total[df_total['cluster_id'].isin([1, 2])]
df

,검정연도,cluster_id,손해율(true),손해율(pred)
1,2017,1,4.033635,0.107160
2,2017,2,1.540479,1.581532
8,2018,1,1.218203,0.000000
9,2018,2,0.000000,1.367975
15,2019,1,2.555051,1.687864
16,2019,2,4.549462,6.261528
22,2020,1,0.000000,1.472917
23,2020,2,0.000000,0.360544


In [9]:
from collections import defaultdict

cases = defaultdict(dict)
years = [2017, 2018, 2019, 2020]

def pick_cluster(idx, items=[], n_pick=0):
    if n_pick == 3:
        cases[tuple(items)] = {}
        for year in years:
            df = df_total[df_total['cluster_id'].isin(items)]
            df = df[df.검정연도 == year]
            cases[tuple(items)][year] = {
                '손해율(true)_mean': df['손해율(true)'].mean(),
                '손해율(pred)_mean': df['손해율(pred)'].mean(),
                '손해율(true)_std': df['손해율(true)'].std(),
                '손해율(pred)_std': df['손해율(pred)'].std(),
            }
        return
    for i in range(idx + 1, 6 + 1):
        items.append(i)
        pick_cluster(i, items, n_pick + 1)
        items.pop()
        
pick_cluster(-1, items=[], n_pick=0)

In [10]:
for cluster in cases.keys():
    for year, data in cases[cluster].items():
        print(f'{cluster} | {year}')
        print(f"\t손해율(true)_mean: {data['손해율(true)_mean']:.4f}")
        print(f"\t손해율(true)_std : {data['손해율(true)_std']:.4f}")
        print(f"\t손해율(pred)_mean: {data['손해율(pred)_mean']:.4f}")
        print(f"\t손해율(pred)_std : {data['손해율(pred)_std']:.4f}")

(0, 1, 2) | 2017
	손해율(true)_mean: 2.0159
	손해율(true)_std : 1.8271
	손해율(pred)_mean: 1.1823
	손해율(pred)_std : 0.9414
(0, 1, 2) | 2018
	손해율(true)_mean: 1.1423
	손해율(true)_std : 1.1063
	손해율(pred)_mean: 0.8779
	손해율(pred)_std : 0.7620
(0, 1, 2) | 2019
	손해율(true)_mean: 5.2943
	손해율(true)_std : 3.1779
	손해율(pred)_mean: 5.7425
	손해율(pred)_std : 3.8217
(0, 1, 2) | 2020
	손해율(true)_mean: 0.8994
	손해율(true)_std : 1.5579
	손해율(pred)_mean: 0.6503
	손해율(pred)_std : 0.7227
(0, 1, 3) | 2017
	손해율(true)_mean: 2.8206
	손해율(true)_std : 2.0331
	손해율(pred)_mean: 2.2491
	손해율(pred)_std : 2.3617
(0, 1, 3) | 2018
	손해율(true)_mean: 1.8188
	손해율(true)_std : 0.5278
	손해율(pred)_mean: 1.2118
	손해율(pred)_std : 1.1857
(0, 1, 3) | 2019
	손해율(true)_mean: 3.7779
	손해율(true)_std : 4.5152
	손해율(pred)_mean: 3.6553
	손해율(pred)_std : 4.9421
(0, 1, 3) | 2020
	손해율(true)_mean: 0.8994
	손해율(true)_std : 1.5579
	손해율(pred)_mean: 0.5301
	손해율(pred)_std : 0.8186
(0, 1, 4) | 2017
	손해율(true)_mean: 2.7487
	손해율(true)_std : 1.9759
	손해율(pred)_mean: 1.8447
	손해율(pr

In [11]:
min_std_true, min_mean_true = {'cluster': -1, 'value': float('inf'), 'year': -1}, {'cluster': -1, 'value': float('inf'), 'year': -1}
min_std_pred, min_mean_pred = {'cluster': -1, 'value': float('inf'), 'year': -1}, {'cluster': -1, 'value': float('inf'), 'year': -1}

for cluster in cases.keys():
    for year, data in cases[cluster].items():
        if data['손해율(true)_mean'] < min_mean_true['value']:
            min_mean_true['value'] = data['손해율(true)_mean']
            min_mean_true['cluster'] = cluster
            min_mean_true['year'] = year
        
        if data['손해율(true)_std'] < min_std_true['value']:
            min_std_true['value'] = data['손해율(true)_std']
            min_std_true['cluster'] = cluster
            min_std_true['year'] = year
            
        if data['손해율(pred)_mean'] < min_mean_pred['value']:
            min_mean_pred['value'] = data['손해율(pred)_mean']
            min_mean_pred['cluster'] = cluster
            min_mean_pred['year'] = year
            
        if data['손해율(pred)_std'] < min_std_pred['value']:
            min_std_pred['value'] = data['손해율(pred)_std']
            min_std_pred['cluster'] = cluster
            min_std_pred['year'] = year

In [12]:
print(min_mean_true)
print(min_mean_pred)
print(min_std_true)
print(min_std_pred)

{'cluster': (1, 2, 3), 'value': 0.0, 'year': 2020}
{'cluster': (1, 5, 6), 'value': 0.0, 'year': 2018}
{'cluster': (1, 2, 3), 'value': 0.0, 'year': 2020}
{'cluster': (1, 5, 6), 'value': 0.0, 'year': 2018}


In [13]:
cases_v2 = defaultdict(dict)

def pick_cluster_v2(idx, items=[], n_pick=0):
    if n_pick == 3:
        df = df_total[df_total['cluster_id'].isin(items)]
        cases_v2[tuple(items)] = {
            '손해율(true)_mean': df['손해율(true)'].mean(),
            '손해율(pred)_mean': df['손해율(pred)'].mean(),
            '손해율(true)_std': df['손해율(true)'].std(),
            '손해율(pred)_std': df['손해율(pred)'].std(),
        }
        return
    for i in range(idx + 1, 6 + 1):
        items.append(i)
        pick_cluster_v2(i, items, n_pick + 1)
        items.pop()
        
pick_cluster_v2(-1, items=[], n_pick=0)

In [14]:
min_data = {
    '손해율(true)_mean': {'cluster': -1, 'value': float('inf')},
    '손해율(true)_std': {'cluster': -1, 'value': float('inf')},
    '손해율(pred)_mean': {'cluster': -1, 'value': float('inf')},
    '손해율(pred)_std': {'cluster': -1, 'value': float('inf')},
}

for cluster in cases_v2.keys():
    for k, v in cases_v2[cluster].items():
        if v < min_data[k]['value']:
            min_data[k]['value'] = v
            min_data[k]['cluster'] = cluster
            
min_data

{'손해율(true)_mean': {'cluster': (2, 3, 5), 'value': 1.0061804069973996},
 '손해율(true)_std': {'cluster': (3, 4, 5), 'value': 1.507854676241703},
 '손해율(pred)_mean': {'cluster': (1, 4, 5), 'value': 0.8663413387777769},
 '손해율(pred)_std': {'cluster': (1, 4, 5), 'value': 1.1205369968638101}}

In [19]:
for cluster in cases_v2.keys():
    print(cluster)
    for k, v in cases_v2[cluster].items():
        print(f'\t{k} {v:.4f}')

(0, 1, 2)
	손해율(true)_mean 2.3380
	손해율(pred)_mean 2.1133
	손해율(true)_std 2.5442
	손해율(pred)_std 2.8010
(0, 1, 3)
	손해율(true)_mean 2.3292
	손해율(pred)_mean 1.9116
	손해율(true)_std 2.4934
	손해율(pred)_std 2.7105
(0, 1, 4)
	손해율(true)_mean 2.3776
	손해율(pred)_mean 1.8573
	손해율(true)_std 2.4171
	손해율(pred)_std 2.5646
(0, 1, 5)
	손해율(true)_mean 1.8305
	손해율(pred)_mean 1.3680
	손해율(true)_std 2.5853
	손해율(pred)_std 2.5929
(0, 1, 6)
	손해율(true)_mean 2.5614
	손해율(pred)_mean 2.5342
	손해율(true)_std 2.4292
	손해율(pred)_std 3.7598
(0, 2, 3)
	손해율(true)_mean 2.1861
	손해율(pred)_mean 2.4369
	손해율(true)_std 2.6099
	손해율(pred)_std 2.8933
(0, 2, 4)
	손해율(true)_mean 2.2346
	손해율(pred)_mean 2.3826
	손해율(true)_std 2.5401
	손해율(pred)_std 2.7683
(0, 2, 5)
	손해율(true)_mean 1.6874
	손해율(pred)_mean 1.8933
	손해율(true)_std 2.6688
	손해율(pred)_std 2.8932
(0, 2, 6)
	손해율(true)_mean 2.4183
	손해율(pred)_mean 3.0595
	손해율(true)_std 2.5628
	손해율(pred)_std 3.8009
(0, 3, 4)
	손해율(true)_mean 2.2258
	손해율(pred)_mean 2.1810
	손해율(true)_std 2.4888
	손해율(pred)_std 2.6988
